# Word Sense Disambugation

In [4]:
import sys
import os

os.getcwd()
project_path = os.path.abspath("Processing")
project_path = os.path.abspath("LLM")

if project_path not in sys.path:
    sys.path.append(project_path)

In [5]:
from Processing.script import load_jax_array, unzip_jax_store

In [ ]:
chunk_csv_file = "../Data/Processed/SemCoreChunks/chunk_0.csv"
chunk_jax_store = "../Data/Processed/jax_store/chunk_0.zip"


In [ ]:
python script.py \
    --input-csv ../Data/Processed/SemCoreChunks/chunk_0.csv \
    --output-csv ../Data/Processed/ProcessedSemCoreChunks/chunk_0.csv \
    --chunk-store ../Data/Processed/jax_store/chunk_0 \
    --tokenizer /home/matt/.llama/checkpoints/Llama3.2-1B-hf-tok/tokenizer.model \
    --model-weights ../Data/ModelWeights/llama_jax_weights.pkl
